THIS IS WHERE THE NEW PROJECT STARTS!!!!

In [1]:
# this cell imports all the necessary packages/libraries related to the project
import numpy as np
import scipy 
import json
import pandas as pd
import gzip
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn import model_selection  
from sklearn import linear_model
from sklearn import metrics
from sklearn import tree
from sklearn import neighbors
from sklearn import svm
from sklearn import ensemble
from sklearn import cluster


import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim.models.keyedvectors as word2vec

In [2]:
# model is a pretrained text file that is available for use in the Word2Vec library.
model =  word2vec.KeyedVectors.load_word2vec_format('w2vmodel.bin', binary=True)

In [65]:
sen = ["I", "love", "this", "class"]
sen_emd = np.zeros(300)
for w in sen:
    if w in model:
        sen_emd = np.add(model[w], sen_emd)

In [67]:
#model["I"]

In [64]:
sen_emd

array([ 2.10937500e-01,  1.98089600e-01,  4.54101562e-02,  7.83203125e-01,
       -1.52343750e-01,  2.72827148e-02,  4.12658691e-01, -6.23535156e-01,
       -2.55371094e-01,  4.63378906e-01,  1.30989075e-01, -1.92016602e-01,
        2.57324219e-01, -3.33251953e-01, -5.46875000e-01,  3.65112305e-01,
        2.75878906e-01,  2.42919922e-01,  3.28125000e-01, -3.92089844e-01,
       -8.22753906e-02,  4.84375000e-01,  4.10400391e-01, -4.22119141e-01,
        2.01904297e-01,  5.56335449e-02, -1.12792969e-01,  1.87500000e-01,
        2.04986572e-01,  3.02734375e-02,  4.39453125e-03,  4.11865234e-01,
        1.20056152e-01,  1.14620209e-01,  2.81127930e-01,  1.67411804e-01,
        1.57348633e-01,  1.34277344e-01,  1.36047363e-01,  6.85058594e-01,
        3.66699219e-01, -2.71911621e-01,  4.56054688e-01,  1.78344727e-01,
        2.23632812e-01,  9.44213867e-02,  3.61511230e-01,  3.77197266e-01,
        2.11791992e-02,  1.09130859e-01,  2.44140625e-03,  2.93457031e-01,
       -1.09619141e-01, -

In [5]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

In [6]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield json.dumps(eval(l))

f = open("output.strict", 'w')
for l in parse("reviews_Kindle_Store_5.json.gz"):
    f.write(l + '\n')

In [7]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Kindle_Store_5.json.gz')

In [8]:
#df2=df
df2 = df[:1000]

In [9]:
del df2['reviewerID']
del df2['reviewerName']
del df2['summary']
del df2['helpful']
del df2['unixReviewTime']
del df2['reviewTime']

In [10]:
df2.head()

,asin,reviewText,overall
0,B000F83SZQ,I enjoy vintage books and movies so I enjoyed ...,5.0
1,B000F83SZQ,This book is a reissue of an old one; the auth...,4.0
2,B000F83SZQ,This was a fairly interesting read. It had ol...,4.0
3,B000F83SZQ,I'd never read any of the Amy Brewster mysteri...,5.0
4,B000F83SZQ,"If you like period pieces - clothing, lingo, y...",4.0


In [11]:
df2.shape

(1000, 3)

In [12]:
df2= df2.set_index("asin")

In [13]:
df2_overall= pd.get_dummies(df2["overall"])

In [14]:
df2.head()

,reviewText,overall
asin,,
B000F83SZQ,I enjoy vintage books and movies so I enjoyed ...,5.0
B000F83SZQ,This book is a reissue of an old one; the auth...,4.0
B000F83SZQ,This was a fairly interesting read. It had ol...,4.0
B000F83SZQ,I'd never read any of the Amy Brewster mysteri...,5.0
B000F83SZQ,"If you like period pieces - clothing, lingo, y...",4.0


In [15]:
#Creating dummies for the column: overall
df3= pd.concat([df2,df2_overall], axis=1)
df3.head()

,reviewText,overall,1.0,2.0,3.0,4.0,5.0
asin,,,,,,,
B000F83SZQ,I enjoy vintage books and movies so I enjoyed ...,5.0,0,0,0,0,1
B000F83SZQ,This book is a reissue of an old one; the auth...,4.0,0,0,0,1,0
B000F83SZQ,This was a fairly interesting read. It had ol...,4.0,0,0,0,1,0
B000F83SZQ,I'd never read any of the Amy Brewster mysteri...,5.0,0,0,0,0,1
B000F83SZQ,"If you like period pieces - clothing, lingo, y...",4.0,0,0,0,1,0


In [16]:
#creating a new column: score with values pre-set as 0
df3["score"]= [0]*len(df3)
df3.head()

,reviewText,overall,1.0,2.0,3.0,4.0,5.0,score
asin,,,,,,,,
B000F83SZQ,I enjoy vintage books and movies so I enjoyed ...,5.0,0,0,0,0,1,0
B000F83SZQ,This book is a reissue of an old one; the auth...,4.0,0,0,0,1,0,0
B000F83SZQ,This was a fairly interesting read. It had ol...,4.0,0,0,0,1,0,0
B000F83SZQ,I'd never read any of the Amy Brewster mysteri...,5.0,0,0,0,0,1,0
B000F83SZQ,"If you like period pieces - clothing, lingo, y...",4.0,0,0,0,1,0,0


In [17]:
#adding columns: 4.0 & 5.0. What we get is a column score that has value 1 
#when the overall (rating) is 4 or 5 and 0 otherwise

df3["score"]=df3[4.0]+df3[5.0]

In [90]:
df3

,reviewText,overall,score
asin,,,
B000F83SZQ,i enjoy vintage books and movies so i enjoyed ...,5.0,1
B000F83SZQ,this book is a reissue of an old one the autho...,4.0,1
B000F83SZQ,this was a fairly interesting read it had old...,4.0,1
B000F83SZQ,id never read any of the amy brewster mysterie...,5.0,1
B000F83SZQ,if you like period pieces clothing lingo you ...,4.0,1
B000F83SZQ,a beautiful indepth character description make...,4.0,1
B000F83SZQ,i enjoyed this one tho im not sure why its cal...,4.0,1
B000F83SZQ,never heard of amy brewster but i dont need to...,4.0,1
B000FA64PA,darth maul working under cloak of darkness com...,5.0,1


In [18]:
#cleaning the dataset
del df3[1.0]
del df3[2.0]
del df3[3.0]
del df3[4.0]
del df3[5.0]


In [19]:
df3.reviewText = df3.reviewText.apply(lambda x: x.lower())

In [20]:
import string
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [21]:
df3.reviewText = df3.reviewText.apply(remove_punctuations)


In [22]:
#converting reviews to lists (tokenizing and splitting)
tokenized = df3.reviewText.str.split().tolist()

In [23]:
#print (tokenized)

In [24]:
stop = open("englishstopwords.txt","r+")

In [25]:
stop= stop.read()

In [26]:
stoplist = stop.split()
#print (stoplist)

In [35]:
# tk = [i for i in tokenized if i not in stoplist]
# print(tk)

In [83]:
semd=[]
for item in tokenized:
    wemd=np.zeros(300)
    for w in item:
         if w in model:
                wemd = np.add(model[w],wemd)
    semd.append(wemd)
print(semd)    
                    
                    
                    
                    
# vocab = model.vocab.keys()
# sentence = ["London", "is", "the", "capital", "of", "Great", "Britain"]
# vectors=[]
# for w in sentence:
#     if w in vocab:
#         vectors.append(model[w])
#     else:
#         print("Word {} not in vocab".format(w))
#         vectors.append([0])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [84]:
semd[0].shape

(300,)

In [85]:
len(semd)

1000

In [108]:
semd[2]

IndexError: list index out of range

In [91]:
scorelist = df3["score"].tolist()

In [106]:
#scorelist

In [100]:
def listtodict(scorelist, semd):
    d = {}
    while scorelist:
        d[scorelist.pop()] = semd.pop()
    return d

In [104]:
listtodict(scorelist,semd)

IndexError: pop from empty list